In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
import cv2

import functions as fn

In [2]:
dataDir='../data'
trainDir='train2017'
valDir='val2017'
seed=42
transformed=.5
train=.6
val=.2
test = 1 - train - val
strategy_list = ['left', 'right']

np.random.seed(seed)

trainAnn=f'{dataDir}/annotations/instances_{trainDir}.json'
valAnn=f'{dataDir}/annotations/instances_{valDir}.json'

cocoTrain=COCO(trainAnn)
cocoVal=COCO(valAnn)

catIdsTrain = cocoTrain.getCatIds(catNms=['person'])
imgIdsTrain = cocoTrain.getImgIds(catIds=catIdsTrain)
imgIdsTrain = cocoTrain.getImgIds(imgIds=imgIdsTrain)
annIdsTrain = cocoTrain.getAnnIds(imgIds=imgIdsTrain, catIds=catIdsTrain, iscrowd=None)

catIdsVal = cocoVal.getCatIds(catNms=['person'])
imgIdsVal = cocoVal.getImgIds(catIds=catIdsVal)
imgIdsVal = cocoVal.getImgIds(imgIds=imgIdsVal)
annIdsVal = cocoVal.getAnnIds(imgIds=imgIdsVal, catIds=catIdsVal, iscrowd=None)

loading annotations into memory...
Done (t=10.18s)
creating index...
index created!
loading annotations into memory...
Done (t=0.39s)
creating index...
index created!


In [47]:
count = 0
limit = 10

# 69 as control
# 42 works

idx = 7

testImg = cocoTrain.loadImgs(imgIdsTrain[idx])[0]
testAnn = cocoTrain.loadAnns(cocoTrain.getAnnIds(imgIds=testImg['id'], catIds=catIdsTrain, iscrowd=None))

fpath = '{}/{}/{}'.format(dataDir, trainDir, testImg['file_name'])

testImgArr = cv2.imread(fpath)

fn.remove_true_blacks(testImgArr)

whichAnn = np.random.randint(0, len(testAnn))

testSpecAnn = testAnn[whichAnn] # np.random.randint(0, len(testAnn))
strategy = strategy_list[np.random.randint(0, len(strategy_list))]

strategy='right'

print(whichAnn, strategy)

maskedTest, maskedTestInv = fn.generate_masked_images(testImgArr, testSpecAnn)

newMaskedTest = fn.apply_rolling_shutter(maskedTest, testSpecAnn, strategy=strategy)

0 right


In [5]:
fn.display_image(testImgArr)

In [11]:
fn.get_ann_dims(testSpecAnn)

(23, 175, 254, 357)

In [25]:
testImgArr.shape

(640, 427, 3)

In [51]:
def fill_inv_masked(inv_masked_img, ann, strategy='right'): # when strategy='right', pull from left; when strategy='left', pull from right
    output = np.copy(inv_masked_img)

    xmin, ymin, width, height = fn.get_ann_dims(ann)
    id = ann['id']
    atLeft, atRight = False, False

    if strategy not in ['left', 'right']:
        raise ValueError(f'Strategy {strategy} is invalid')

    if xmin == 0: # at left side of image
        atLeft = True
    if xmin + width == inv_masked_img.shape[1]: # at right side of image
        atRight = True
    
    if atLeft:
        if atRight:
            raise ValueError(f'Image #{id} takes up entire image width, cannot fill')
        else:
            strategy='left'
    elif atRight:
        strategy='right'

    if strategy == 'right': # pull from left
        for row in np.arange(ymin, ymin + height):
            blacks = np.sort(np.array(list(set(np.where(output[row] == np.array([0, 0, 0]))[0]))))
            if(len(blacks) > 0):
                low = 2 * blacks[0] - 1 - blacks[-1]
                high = blacks[0]
                foo = np.flip(output[row][low:high], axis=0)
                if(low < 0):
                    foo = np.pad(np.flip(maskedTestInv[row][0:high], axis=0), ((0, -low), (0, 0)), mode='reflect')
                output[row][blacks[0]:blacks[-1] + 1] = foo
    else: # pull from right
        for row in np.arange(ymin, ymin + height):
            blacks = np.sort(np.array(list(set(np.where(output[row] == np.array([0, 0, 0]))[0]))))
            if(len(blacks) > 0):
                low = blacks[-1] + 1
                high = 2 * blacks[-1] + 2 - blacks[0] 
                foo = np.flip(output[row][low:high], axis=0)
                if(high > output.shape[1]):
                    foo = np.pad(np.flip(maskedTestInv[row][low:maskedTestInv.shape[1]], axis=0), ((0, high-maskedTestInv.shape[1]), (0, 0)), mode='reflect')
                output[row][blacks[0]:blacks[-1] + 1] = foo
    return output

newMaskedTestInv = fill_inv_masked(maskedTestInv, testSpecAnn, strategy=strategy)

final_output = fn.recombine_masked_imgs(newMaskedTest, newMaskedTestInv)

fn.display_image(final_output)

In [21]:
fn.display_image(testImgArr)

In [43]:
row = 265

blacks = np.sort(np.array(list(set(np.where(maskedTestInv[row] == np.array([0, 0, 0]))[0]))))
if(len(blacks) > 0):
    low = blacks[-1] + 1
    high = 2 * blacks[-1] + 2 - blacks[0] 
    foo = np.flip(maskedTestInv[row][low:high], axis=0)
    if(high > maskedTestInv.shape[1]):
        foo = np.pad(np.flip(maskedTestInv[row][low:maskedTestInv.shape[1]], axis=0), ((0, high-maskedTestInv.shape[1]), (0, 0)), mode='reflect')
        print(len(foo))

165
